Resultados esperados:
 - input: 
    - avg_runtime
    - input_complexity
    - speedup
    - eficiencia
 - output: 
    - p_cores


In [1]:
import pandas as pd
import tensorflow as tf
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error

In [2]:
df_clean = pd.read_csv("dataset/clean_data.csv")

# df_clean['cores'] = df_clean['cores']/df_clean['cores'].max()
df_clean['input'] = df_clean['input']/df_clean['input'].max()

In [3]:
df_clean.columns

Index(['cores', 'input', 'average_runtime', 'speedup', 'efficiency', 'mem_ram',
       'freq_mem', 'freq_cpu', 'mem_l3'],
      dtype='object')

In [4]:
X = df_clean[['cores', 'input', 'average_runtime', 'speedup', 'mem_ram', 'freq_mem', 'freq_cpu', 'mem_l3']].values
y = df_clean['efficiency'].values

### Regression Models

#### MLP Regression Model

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)

In [17]:
learning_rate = 0.00001
input_shape = (X_train.shape[1],)

# callback = EarlyStopping(monitor="loss", patience=20)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(16, activation='relu', input_shape = input_shape))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(X_train, y_train, batch_size=32, epochs=1000, verbose=2, validation_data=(X_test, y_test))

Epoch 1/1000
52/52 - 0s - loss: 4816.7241 - val_loss: 234.2727 - 492ms/epoch - 9ms/step
Epoch 2/1000
52/52 - 0s - loss: 55.0976 - val_loss: 23.8562 - 57ms/epoch - 1ms/step
Epoch 3/1000
52/52 - 0s - loss: 33.3458 - val_loss: 21.4865 - 57ms/epoch - 1ms/step
Epoch 4/1000
52/52 - 0s - loss: 31.2344 - val_loss: 20.2690 - 56ms/epoch - 1ms/step
Epoch 5/1000
52/52 - 0s - loss: 29.8565 - val_loss: 20.2046 - 56ms/epoch - 1ms/step
Epoch 6/1000
52/52 - 0s - loss: 28.8013 - val_loss: 20.5285 - 61ms/epoch - 1ms/step
Epoch 7/1000
52/52 - 0s - loss: 28.2509 - val_loss: 19.7860 - 55ms/epoch - 1ms/step
Epoch 8/1000
52/52 - 0s - loss: 27.6600 - val_loss: 20.4573 - 54ms/epoch - 1ms/step
Epoch 9/1000
52/52 - 0s - loss: 27.4249 - val_loss: 19.2393 - 55ms/epoch - 1ms/step
Epoch 10/1000
52/52 - 0s - loss: 27.4789 - val_loss: 19.7518 - 53ms/epoch - 1ms/step
Epoch 11/1000
52/52 - 0s - loss: 26.9221 - val_loss: 18.8918 - 56ms/epoch - 1ms/step
Epoch 12/1000
52/52 - 0s - loss: 26.5874 - val_loss: 18.4375 - 56ms/ep

In [15]:
loss = model.evaluate(X_test, y_test)
print(f'Erro no conjunto de teste: {loss}')

predictions = model.predict(X_test)

for i in range(10):
    print(f'Valor Real: {y_test[i]}, Previsão: {predictions[i][0]}')

23/23 [==============================] - 0s 591us/step - loss: 33.0391
Erro no conjunto de teste: 33.03910827636719
23/23 [==============================] - 0s 545us/step
Valor Real: 0.1421652660989802, Previsão: 1.9458043575286865
Valor Real: 0.1909549526707994, Previsão: 2.3346974849700928
Valor Real: 0.1362309885647307, Previsão: 2.1098363399505615
Valor Real: 0.0688319593505949, Previsão: 1.8008458614349365
Valor Real: 0.1343610885294778, Previsão: 2.3414418697357178
Valor Real: 0.1712759335286006, Previsão: 2.2956502437591553
Valor Real: 0.1254793398850434, Previsão: 1.6889684200286865
Valor Real: 0.1133989266295716, Previsão: 2.147601842880249
Valor Real: 0.1366358050892372, Previsão: 2.1123769283294678
Valor Real: 0.1045667369935413, Previsão: 1.7712743282318115


#### MLP Regression Model with Leave-One-Out-Validation (LOOCV)

In [12]:
loo = LeaveOneOut()
mse_scores = []
# callback = EarlyStopping(monitor="loss", patience=10)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(16, activation='relu', input_shape=input_shape))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='linear'))


for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    learning_rate = 0.001
    input_shape = (X_train.shape[1],)

    model.compile(optimizer='adam', loss='mean_squared_error')

    model.fit(X_train, y_train, epochs=100, verbose=0)

    predictions = model.predict(X_test)

    mse = mean_squared_error(y_test, predictions)
    mse_scores.append(mse)

print(f'Erro médio quadrático usando LOOCV: {min(mse_scores)}')

1/1 [==============================] - 0s 35ms/step
Erro médio quadrático usando LOOCV: 4.475726450791921e-08


In [13]:
PATH_MODEL_SAVE = "model/pascal_model.keras"

model.save(PATH_MODEL_SAVE)